In [1]:
##############################################################
# NbIT functions to calculate total inter-correlation coefficient
# via summing entropy over every ATOM, instead of RESIDUE.
# May not be compatible with other NbIT measurements.
# For formulae of all the functions, refer to:
#
# "Spontaneous Inward Opening of the Dopamine Transporter Is 
# Triggered by PIP2‐Regulated Dynamics of the N‐Terminus"
# Khelashvili et al. ACS Chemical Neuroscience (2015)
# 
# Hao Wu, Ph.D.
# Prof. Harel Weinstein's lab
# Weill Cornell Medicine
# haw4005@med.cornell.edu
# Last Updated: Sep 02 2021
##############################################################

##############################################################
# DEFINE FUNCTIONS
##############################################################

import MDAnalysis as mda
import numpy as np
#import matplotlib.pyplot as plt
from math import pi, log
#import pandas as pd

# Compute COV matrix in a sliding window manner
def compute_cov_matrix_sliding_window(atoms, atoms_ref, win_start, win_end, nsteps):
    p0 = atoms_ref.positions
    p0.shape
    # Compute the distance matrices
    N = (win_end-win_start)//nsteps
    dist_matrix = np.empty((p0.shape[0], nsteps, N))
    for n in range(N):
        for ts in u.trajectory[n*nsteps+win_start:(n+1)*nsteps+win_start]:
            dist_matrix[:, ts.frame-nsteps*n-win_start, n] = np.linalg.norm(atoms.positions-p0, axis = -1)[:]
    # Compute covariance matrix
    cov1 = np.empty((p0.shape[0], p0.shape[0], N))
    for n in range(N):
        cov1 [:, :, n] = np.cov(dist_matrix[:, :, n], rowvar=True)
    # Compute the averaged covariance matrix
    COV = np.average(cov1, axis = -1)
    return COV

def entropy(i):
    # i: atom indices in original trajectory
    # j: atom indices in a trajectory excluding hydrogen atoms, equals the first two dimensions of COV matrix
    j = np.where(np.in1d(atoms_ref.ids.tolist(), i)==True)[0].tolist()
    # if len(j)>1: H is the entropy of a site containing more than one atom
    if len(j)>1:
        H = 0.5*len(j)*(1+log(2*pi))+0.5*np.linalg.slogdet(np.take(np.take(COV, j, axis = 0), j, axis = -1))[1]
        return H
    # if len(j)==1: H is the entropy of a single-atom site
    elif len(j)==1:
        H = 0.5*len(j)*(1+log(2*pi))+0.5*log(np.take(np.take(COV, j, axis = -1), j, axis = 0))
        return H
    else:
        return print('Wrong index format or no elements in the list were found')

# total correlation: sum H over every atom
def total_correlation_by_atom(i):
    sum_H = 0
    for atom_id in i:
        sum_H += entropy(atom_id)
    H = entropy(i)
    return sum_H - H

def conditional_entropy(i, j):
    k = np.setdiff1d(i, j, assume_unique=True)
    Hij = entropy(np.hstack((k, j)))
    Hj = entropy(j)
    return Hij - Hj

def conditional_total_correlation_by_atom(i, j):
    i = np.setdiff1d(i, j, assume_unique=True).tolist()
    sum_H = 0
    for atom_id in i:
        sum_H += conditional_entropy(atom_id, j)
    H = conditional_entropy(i, j)
    return sum_H - H

def total_inter_correlation_by_atom(i, j):
    k = np.hstack((i, j)).tolist()
    return (total_correlation_by_atom(k) - conditional_total_correlation_by_atom(i, j) - conditional_total_correlation_by_atom(j, i))

def total_inter_correlation_coeff_by_atom(i, j):
    # dimension = total # of atoms in i and j minus 1
    scaling_coeff = len(i) + len(j) - 1 
    exp_part = np.exp(-2*total_inter_correlation_by_atom(i, j)/scaling_coeff)
    return np.sqrt(1 - exp_part)

In [2]:
##############################################################
# LOAD TRAJECTORY AND DEFINE SITES
##############################################################

# Load the trajectory assuming it is already aligned
# by Ambrose's autoalign code: https://github.com/weinsteinlab/Auto-align
u = mda.Universe('./benserazide_S.psf',
                 './benserazide_S_all_poses_prot_lig_wrapped_lig_rmsd_lt_4_aligned.xtc')
# Load the reference structure
u_ref = mda.Universe('./benserazide_S.psf', 
                     './benserazide_S_ref_last_frame.pdb')

total_n_frames = u.trajectory.n_frames

# Select the regions for NbIT
# DBS: dimeric binding site, residues within 4 angstroms of ligand in at least 40% of the large subset 
DBS = u.select_atoms('((segid PROA and resid 170 172 138 166 137 171) or (segid PROB and resid 2 214 1 216)) and not type H')
# Occluding residues in apo and holo simulations: 
OR_apo = u.select_atoms('(segid PROA and resid 163 146 144 43 42 40 39 27 38 164 28 143 147 44) and not type H')
OR_holo = u.select_atoms('(segid PROA and resid 164 163 147 146 144 143 43 42 40 39 38 28 27 165 44) and not type H')
# three dummy regions (one in each domain) selected as references
D1_A = u.select_atoms('(segid PROA and resid 69 70 71 72 73 74 75) and not type H')
D2_A = u.select_atoms('(segid PROA and resid 151 152 153 154 155 156 157) and not type H')
D3_A = u.select_atoms('(segid PROA and resid 235 236 237 238 239 240 241) and not type H')
D1_B = u.select_atoms('(segid PROB and resid 69 70 71 72 73 74 75) and not type H')
D2_B = u.select_atoms('(segid PROB and resid 151 152 153 154 155 156 157) and not type H')
D3_B = u.select_atoms('(segid PROB and resid 235 236 237 238 239 240 241) and not type H')

# Select combinations of regions for the analysis
COMB = [DBS, OR_apo, OR_holo, D1_A, D2_A, D3_A, D1_B, D2_B, D3_B]
names = ['DBS', 'OR_apo', 'OR_holo', 'D1_A', 'D2_A', 'D3_A', 'D1_B', 'D2_B', 'D3_B']

# Select the atoms for covariance matrix calculations
atoms = u.select_atoms('not type H')
atoms_ref = u_ref.select_atoms('not type H')

/Users/haw4005/opt/anaconda3/envs/mdanalysis/lib/python3.8/site-packages/MDAnalysis/lib/mdamath.py:247: RuntimeWarning: invalid value encountered in double_scalars
  alpha = np.rad2deg(np.arccos(np.dot(y, z) / (ly * lz)))
/Users/haw4005/opt/anaconda3/envs/mdanalysis/lib/python3.8/site-packages/MDAnalysis/lib/mdamath.py:248: RuntimeWarning: invalid value encountered in double_scalars
  beta = np.rad2deg(np.arccos(np.dot(x, z) / (lx * lz)))
/Users/haw4005/opt/anaconda3/envs/mdanalysis/lib/python3.8/site-packages/MDAnalysis/lib/mdamath.py:249: RuntimeWarning: invalid value encountered in double_scalars
  gamma = np.rad2deg(np.arccos(np.dot(x, y) / (lx * ly)))


In [5]:
##############################################################
# CALCULATE TIC COEFFICIENT IN SLIDING WINDOWS
##############################################################

# Define window size and separation
win_size = 500 # 40 ns
win_separation = 125 # 10 ns 
# Set the number of steps in for covariance matrix averaging frame (<= win_size)
nsteps = win_size
win_num = 0
for win_start in np.arange(0, total_n_frames, win_separation):
    win_end = win_start+win_size
    if win_end > total_n_frames:
        break
    else:
        win_num += 1 

site0 = DBS.ids.tolist()
site1 = OR_apo.ids.tolist()
site2 = OR_holo.ids.tolist()

win_start_series = np.zeros(win_num)
TIC_coeff1 = np.zeros(win_num)
TIC_coeff2 = np.zeros(win_num)

i = 0
for win_start in np.arange(0, total_n_frames, win_separation):
    win_end = win_start+win_size
    if win_end > total_n_frames:
        break

    win_start_series[i] = win_start

    print("Sliding window {} - {}:".format(win_start, win_end))
    COV = compute_cov_matrix_sliding_window(atoms, atoms_ref, win_start, win_end, nsteps)

    TIC_coeff1[i] = total_inter_correlation_coeff_by_atom(site0, site1)
    print("DBS-OR_apo TIC coeff = {:.4f}".format(TIC_coeff1[i]))

    TIC_coeff2[i] = total_inter_correlation_coeff_by_atom(site0, site2)
    print("DBS-OR_holo TIC coeff = {:.4f}".format(TIC_coeff2[i]))

    i += 1
    
file1 = "./TIC_coeff_DBS_OR_apo_OR_holo.txt"
np.savetxt(file1, np.c_[win_start_series, TIC_coeff1, TIC_coeff2], 
    fmt=["%d", "%.4f", "%.4f"], header="win_start DBS-OR_apo DBS-OR_holo")

Sliding window 0 - 500:
DBS-OR_apo TIC coeff = 0.5326
DBS-OR_holo TIC coeff = 0.5486
Sliding window 125 - 625:
DBS-OR_apo TIC coeff = 0.5659
DBS-OR_holo TIC coeff = 0.6063
Sliding window 250 - 750:
DBS-OR_apo TIC coeff = 0.5889
DBS-OR_holo TIC coeff = 0.6372
Sliding window 375 - 875:
DBS-OR_apo TIC coeff = 0.5973
DBS-OR_holo TIC coeff = 0.6236
Sliding window 500 - 1000:
DBS-OR_apo TIC coeff = 0.5823
DBS-OR_holo TIC coeff = 0.5822
Sliding window 625 - 1125:
DBS-OR_apo TIC coeff = 0.5806
DBS-OR_holo TIC coeff = 0.5874
Sliding window 750 - 1250:
DBS-OR_apo TIC coeff = 0.5579
DBS-OR_holo TIC coeff = 0.5755


In [4]:
##############################################################
# CALCULATE TIC COEFFICIENT IN THE ENTIRE TRAJECTORY
##############################################################

win_start = 0
win_end = total_n_frames
nsteps = win_end - win_start
COV = compute_cov_matrix_sliding_window(atoms, atoms_ref, win_start, win_end, nsteps)

site0 = DBS.ids.tolist()
site1 = OR_apo.ids.tolist()
site2 = OR_holo.ids.tolist()
site3 = D1_A.ids.tolist()
site4 = D2_A.ids.tolist()
site5 = D3_A.ids.tolist()
site6 = D1_B.ids.tolist()
site7 = D2_B.ids.tolist()
site8 = D3_B.ids.tolist()

TIC_coeff1 = total_inter_correlation_coeff_by_atom(site0, site1)
print("DBS-OR_apo TIC coeff = {:.4f}".format(TIC_coeff1))

TIC_coeff2 = total_inter_correlation_coeff_by_atom(site0, site2)
print("DBS-OR_holo TIC coeff = {:.4f}".format(TIC_coeff2))

TIC_coeff3 = total_inter_correlation_coeff_by_atom(site0, site3)
print("DBS-D1_A TIC coeff = {:.4f}".format(TIC_coeff3))

TIC_coeff4 = total_inter_correlation_coeff_by_atom(site0, site4)
print("DBS-D2_A TIC coeff = {:.4f}".format(TIC_coeff4))

TIC_coeff5 = total_inter_correlation_coeff_by_atom(site0, site5)
print("DBS-D3_A TIC coeff = {:.4f}".format(TIC_coeff5))

TIC_coeff6 = total_inter_correlation_coeff_by_atom(site0, site6)
print("DBS-D1_B TIC coeff = {:.4f}".format(TIC_coeff6))

TIC_coeff7 = total_inter_correlation_coeff_by_atom(site0, site7)
print("DBS-D2_B TIC coeff = {:.4f}".format(TIC_coeff7))

TIC_coeff8 = total_inter_correlation_coeff_by_atom(site0, site8)
print("DBS-D3_B TIC coeff = {:.4f}".format(TIC_coeff8))

f = open("./TIC_entire_traj_test.txt", "w")
f.write("OR_apo {:.4f}\n".format(TIC_coeff1))
f.write("OR_holo {:.4f}\n".format(TIC_coeff2))
f.write("D1_A {:.4f}\n".format(TIC_coeff3))
f.write("D2_A {:.4f}\n".format(TIC_coeff4))
f.write("D3_A {:.4f}\n".format(TIC_coeff5))
f.write("D1_B {:.4f}\n".format(TIC_coeff6))
f.write("D2_B {:.4f}\n".format(TIC_coeff7))
f.write("D3_B {:.4f}\n".format(TIC_coeff8))
f.close()

DBS-OR_apo TIC coeff = 0.5266
DBS-OR_holo TIC coeff = 0.5623
DBS-D1_A TIC coeff = 0.3640
DBS-D2_A TIC coeff = 0.3586
DBS-D3_A TIC coeff = 0.4217
DBS-D1_B TIC coeff = 0.3288
DBS-D2_B TIC coeff = 0.3853
DBS-D3_B TIC coeff = 0.4524
